## Scrape Hawaii City Names

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd

In [2]:
# URL to scrape
url = 'https://www.countryaah.com/alphabetical-list-of-cities-and-towns-in-hawaii/'

# Retrieve page with the requests module
html = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')

In [3]:
table_data = soup.find(class_='entry-content').find_all('ul')[0]

In [4]:
cities_ls = []

for data in table_data:
    try:
        if data.text[-1] == 'I':
            cities_ls.append(data.text[:-4])
    except:
        continue


## Call OpenWeatherAPI and Store Weather Data in sqlite

In [5]:
from config import api_key
import json
import requests

url = "http://api.openweathermap.org/data/2.5/weather?" + "appid=" + api_key + "&q="

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float


class WeatherReading(Base):
    __tablename__ = 'weather'
    city_name = Column(String(255), primary_key=True)
    main_weather = Column(String(255))
    temperature = Column(Float)
    humidity = Column(Float)
    cloudiness = Column(Float)
    wind_speed = Column(Float)

engine = create_engine("sqlite:///weather.sqlite")
conn = engine.connect()



In [7]:
weather_ls = []
for city in cities_ls:
    query_url = url + city+ "&units=imperial"
    weather_json = requests.get(query_url).json()
    try:
        print(f"Processing {weather_json['name']}")
        weather_reading = WeatherReading(
            city_name = city,
            main_weather = weather_json['weather'][0]['main'],
            temperature = round(weather_json['main']['temp'], 1),
            humidity = weather_json['main']['humidity'],
            cloudiness = weather_json['clouds']['all'],
            wind_speed = weather_json['wind']['speed']
        )
        weather_ls.append(weather_reading)
    except Exception as e:
        print(e)


Processing ‘Aiea
Processing Anahola
Processing Captain Cook
Processing Eleele
Processing ‘Ewa Beach
Processing Hale‘iwa
Processing Hana
Processing Hanalei
Processing Hanamaulu
Processing Hanapēpē
Processing Hau‘ula
Processing Hāwī
Processing Hilo
Processing Holualoa
'name'
Processing Honoka‘a
Processing Honolulu
Processing Honomu
Processing Kaaawa
Processing Kahuku
Processing Kahului
Processing Kailua
Processing Kalāheo
Processing Kaneohe
Processing Kapaa
Processing Kapaau
'name'
Processing Kaunakakai
Processing Kea‘au
Processing Kealakekua
Processing Kekaha
Processing Kīhei
Processing Kilauea
Processing Koloa
Processing Kualapu‘u
Processing Kurtistown
Processing Lahaina
Processing Lā‘ie
Processing Lanai City
'name'
Processing Lawai
Processing Lihue
Processing Makawao
Processing Maunaloa
Processing Mililani
Processing Mountain View
Processing Naalehu
'name'
Processing Pāhala
'name'
Processing Paia
Processing Pāpa‘ikou
Processing Pearl City
Processing Pepeekeo
Processing Princeville
Pro

In [8]:
# store data in sqlite
from sqlalchemy.orm import Session
Base.metadata.create_all(conn)

session = Session(bind=engine)
for item in weather_ls:
    session.add(item)
session.commit()



session.close()



In [9]:
Base.metadata.create_all(conn)

session = Session(bind=engine)
df = pd.read_sql_query("SELECT * from weather", conn)
session.close()
df.head()


,city_name,main_weather,temperature,humidity,cloudiness,wind_speed
0,Aiea,Clouds,77.3,72.0,40.0,9.22
1,Anahola,Clouds,75.8,76.0,20.0,14.97
2,Captain Cook,Rain,74.3,85.0,71.0,4.81
3,Eleele,Clouds,77.5,71.0,88.0,11.77
4,Ewa Beach,Clear,77.9,67.0,0.0,9.22


In [11]:
#df.to_html("weather_data.html", index=False)